<a href="https://colab.research.google.com/github/LeeJaeEun0/PlayData_230119/blob/main/230119_ch06_DL_05_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seq2Seq
* 문장을 입력 받아 문장을 출력

* 토큰은 단어를 고유한 숫자로 바꿔 놓은 것
* 모든 문장을 사용 -> 다른 문장을 만들어내는 것

어텐션 메커니즘

GRU의 구조

어텐션 디코더
추출된 특성을 밀집 표현으로 넣음

어텐션 기계 번역기

# 데이터 불러오기

In [ ]:
# Tab-delimited Bilingual Sentence Pairs
# 출처 : http://www.manythings.org/anki
# https://github.com/bigdata-young/ai_26th/raw/main/data_dl/corpus.txt # 파일 단위라서 !wget 사용, 옆에 파일이 생긴 것 확인가능
!wget https://github.com/bigdata-young/ai_26th/raw/main/data_dl/corpus.txt

--2023-01-19 08:24:07--  https://github.com/bigdata-young/ai_26th/raw/main/data_dl/corpus.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bigdata-young/ai_26th/main/data_dl/corpus.txt [following]
--2023-01-19 08:24:08--  https://raw.githubusercontent.com/bigdata-young/ai_26th/main/data_dl/corpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253511 (248K) [text/plain]
Saving to: ‘corpus.txt’

corpus.txt          100%[===================>] 247.57K  --.-KB/s    in 0.03s   

2023-01-19 08:24:08 (7.91 MB/s) - ‘corpus.txt’ saved [253511/253511]



In [ ]:
# 텍스트 파일을 전처리 해서 저장

import string # punctuation

l = [] # 특수 문자를 지운 문장들을 받아줄 리스트

# with -> torch.no_grad() => 특정한 객체가 생성되었을때 with구문이 끝나면 close 반환
with open('./corpus.txt', 'r', encoding="utf-8") as f :# open으로 파일 객체가 생성 # 한글은 인코딩 # as는 파일에 이름 f를 붙임
    # open(경로, 'r', encoding=인코딩방식): 파이을 읽어와줌(텍스트 파일)
    # with ... -> 특정한 객체를 생성시키고, with 구문이 끝나면 해당 객체를 삭제(변환)
    # open () as f -> open을 통해 읽어들여온 파일을 f라는 이름의 변수에 할당
    
    lines = f.read().split('\n') # '\n' = 엔터 = 개행문자
    # 파일을 읽어온 다음에 엔터(줄) 기준으로 쪼개줘라 => 문장별로 리스트화
    # lines = ['...', '...', '문장...']
    for line in lines: # 문장
        # 특수문자를 지우고 모든 글자를 소문자로 변경
        # txt = (v for v in line if not v in string.punctuation) # 문자로 
        txt = "".join(v for v in line if not v in string.punctuation).lower() # 문장으로
        l.append(txt)

In [ ]:
l[:5] #\t는 tab

['go\t가', 'hi\t안녕', 'run\t뛰어', 'run\t뛰어', 'who\t누구']

#  학습용 데이터 만들기

* 단어가 10개를 넘지 않는 문장들만 사용
* 문장을 불러올 때 <EOS(End of Speech)> 토큰을 추가해서 문장이 끝났음을 알림

## BOW를 만드는 함수 정의

In [ ]:
import numpy as np
import torch

from torch.utils.data.dataset import Dataset

In [ ]:
def get_BOW(corpus): # 말뭉치 -> 문장 -> BOW를 만드는 함수
    BOW = {"<SOS>": 0, "<EOS>":1}
    # BOW 안에 문장의 시작과 끝을 알리는
    # SOS(Start Of Speech) 토큰과 EOS(End Of Speech) 토큰을 추가

    # 문장 내 단어들을 사용하여 BOW를 생성
    for line in corpus:
        for word in line.split():
            if word not in BOW.keys(): # 등록되지 않은 단어면
                BOW[word] = len(BOW.keys())
                # 사전에 추가해주는데, 해당 단어의 고유번호는 이전까지의 키의 갯수
    return BOW

## 학습용 데이터셋 정의

In [ ]:
class Eng2Kor(Dataset):
    def __init__(self, path='./corpus.txt') -> None:
        super().__init__()
        self.eng_corpus = [] # 영어문장이 들어가는 변수
        self.kor_corpus = [] # 한글문장이 들어가는 변수

        with open(path, 'r', encoding='utf-8') as f:
            lines = f.read().split('\n')
            for line in lines: # 문장
                txt = "".join(v for v in line
                              if not v in string.punctuation).lower()
                # \t 구분이 되어 있었음 (영어와 한글) -> 탭을 기준으로 분리
                engtxt, kortxt = txt.split('\t') # 0 : 영어 # 1 : 한글
                # engtxt = txt.split('\t')[0]
                # kortxt = txt.split('\t')[1]

                # 길이가 10 이하인 문장 = 단어의 갯수가 10개 이하인 문장만 학습
                if len(engtxt.split()) <= 10 and len(kortxt.split()) <= 10:
                    # 영어, 한글 번역문 모두 10개 단어 이하인 데이터만 사용
                    self.eng_corpus.append(engtxt)
                    self.kor_corpus.append(kortxt)
        
        # 영어와 한글 문장을 각각 BOW(단어 사전)으로 변환
        self.engBOW = get_BOW(self.eng_corpus)
        self.korBOW = get_BOW(self.kor_corpus)
    
    # 문장을 단어별로 분리하는 함수
    def gen_seq(self, line): # line = 문장
        seq = line.split() # 토큰화 한다음에
        seq.append("<EOS>") # 마지막에 EOS(문장 끝) 토큰 추가
        return seq

    def __len__(self): # 데이터의 개수를 반환하는 함수
        return len(self.eng_corpus)

    # 데이터와 정답을 반환하는 함수
    def __getitem__(self, i): # data, label을 지정
        # 문자열로 되어 있는 문장을 숫자 표현으로 변경
        # 1) 영어 corpus 중 i번째 문장을 받아옴
        # 2) gen_seq -> i번째 문장을 seq 형태로 변환 (토큰+EOS)
        # 3) 단어 사전을 사용해서 고유번호 형태로 변환 (학습을 위해 숫자형태로 변환)
        data = np.array([
            self.engBOW[txt] for txt in self.gen_seq(self.eng_corpus[i])
        ])
        label = np.array([
            self.korBOW[txt] for txt in self.gen_seq(self.kor_corpus[i])
        ])
        return data, label # 영어 데이터 (입력) -> 한글 데이터 (정답)

## 데이터 로더

In [ ]:
def loader(dataset): # 데이터셋의 문장을 한 문장씩 불러오기 위한 함수 정의
    for i in range(len(dataset)):
        data, label = dataset[i]

        # 데이터와 정답을 반환
        yield torch.tensor(data), torch.tensor(label)
        # yield : 리턴과 유사, 값을 반복적으로 반환

# 모델정의

## 인코더 정의 
* 임베딩층, GRU층

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size) -> None:
        super().__init__()

        # 임베딩층
        self.embedding = nn.Embedding(input_size, hidden_size)
        # GRU층
        self.gru = nn.GRU(hidden_size, hidden_size)
        # nn.GRU : GRU 계산. input_size, hidden_size, num_layers)
    
    def forward(self, x, h): # x: 입력값 / h : 은닉상태
        # 배치 차원과 시계열 차원 추가
        x = self.embedding(x).view(1, 1, -1)
        output, hidden = self.gru(x, h) # output : 문장의 특성, hidden 은닉 상태
        return output, hidden

In [ ]:
# import torch.nn as nn

# class Encoder(nn.Module):
#     def __init__(self, input_size, hidden_size) -> None:
#         super().__init__()

#         # 임베딩층
#         self.embedding = nn.Embedding(input_size, hidden_size)
#         # GRU층
#         self.gru = nn.GRU(hidden_size, hidden_size)
#         #nn.GRU : GRU 계산
#         # (hidden_size, hidden_size, num_layers) -> 아직 아웃풋 사이즐르 모름

#     def forward(self, x, h):
#         # 배치 차원과 시계열 차원 추가
#         x = self.embedding(x).view(1,1,-1) # 단어를 펼침
#         output, hidden = self.gru(x, h) # output: 문장의 특성, hidden: 은닉상태
#         return output, hidden

## 디코더 정의
* 임베딩 층
* 전결합 층 (ReLU)
* 전결합 층 (Softmax)
* 내적
* GRU층

In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=11) -> None:
        super().__init__()

        # 임베딩 층 정의
        self.embedding = nn.Embedding(output_size, hidden_size)

        # 어텐션 가중치를 계산하기 위한 MLP층
        self.attention = nn.Linear(hidden_size * 2, max_length)
        # 10개 + <EOS>(1) = 최대 길이 11개

        # 특징 추출을 위한 MLP층
        self.context = nn.Linear(hidden_size * 2, hidden_size)

        # 오버피팅을 피하기 위한 드롭아웃층
        self.dropout = nn.Dropout(dropout_p)

        # GRU층
        self.gru = nn.GRU(hidden_size, hidden_size)

        # 단어 분류를 위한 MLP층
        self.out = nn.Linear(hidden_size, output_size)

        # 활성화 함수
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)
        # LogSoftmax(dim) : 소프트맥스 함수에 로그 값을 취한 것을 반환
        # dim -> 계산의 대상이 될 차원값
    
    def forward(self, x, h, encoder_outputs): # x : 입력값, h : 은닉상태, e...: 인코더 결과값
        # 입력 받은 x(현 시점의 디코더 입력)을 임베딩 층을 사용해 밀집 표현으로 변환
        x = self.embedding(x).view(1, 1, -1) # 배치 차원, 시계 차원, 단어들.
        x = self.dropout(x)

        # 어텐션 가중치 계산
        attn_weights = self.softmax(
            self.attention(torch.cat((x[0], h[0]), -1))
        )

        # 어텐션 가중치와 인코더의 출력을 내적(크기가 다른 두 배열을 방향이 일치하는 만큼 곱함)
        attn_applied = torch.bmm(
            attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0)
        ) # bmm(A, B) : A 크기가 (B, N, M)이고, B 크기가 (B, M, K)
        # => (B, N, K) 크기의 출력을 반환

        # 인코더 각 시점의 중요도와 밀집 표현을 합쳐서 MLP층으로 특징 추출
        output = torch.cat((x[0], attn_applied[0]), 1)
        output = self.context(output).unsqueeze(0)
        output = self.relu(output)
        # 인코더의 중요도(attn_applied)와 현시점에서의 디코더의 밀집표현(x)을 합쳐서
        # MLP층(context)으로 입력
        # -> MP층은 인코더 각 시점의 중요도와 현시점 디코더의 밀집표현을 동시에 처리
        # -> 인코더의 중요도가 디코더의 반영

        # GRU층으로 입력
        output, hidden = self.gru(output, h)

        # 예측된 단어를 출력
        output = self.out(output[0])

        return output

# 학습 정의

## 학습에 필요한 요소 정의

In [ ]:
import random
from tqdm.notebook import tqdm
from torch.optim.adam import Adam

# 학습에 사용할 프로세서 정의
device = "cuda" if torch.cuda.is_available() else 'cpu'
# 학습에 사용할 데이터셋
dataset = Eng2Kor()

# 인코더 디코더 정의
encoder = Encoder(input_size=len(dataset.engBOW), hidden_size=64).to(device)
decoder = Decoder(64, len(dataset.korBOW), dropout_p=0.1).to(device)
# 인코더와 디코더 학습을 위한 최적화 함수 정의
encoder_optimizer = Adam(encoder.parameters(), lr=0.001)
decoder_optimizer = Adam(decoder.parameters(), lr=0.001)

In [ ]:
device

'cuda'

## 학습 루프 정의

In [ ]:
for epoch in range(200):
    iterator = tqdm(loader(dataset), total=len(dataset))
    total_loss = 0

    for data, label in iterator:
        data = torch.tensor(data, dtype=torch.long).to(device)
        label = torch.tensor(label, dtype=torch.long).to(device)

        # 인코더의 초기 은닉 상태
        encoder_hidden = torch.zeros(1, 1, 64).to(device)
        # 인코더의 모든 시점의 출력을 저장하는 변수
        # 최대 단어 10개 + 종료(EOS) -> 11개
        encoder_outputs = torch.zeros(11, 64).to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        loss = 0

        # 인코더 동작
        for ei in range(len(data)): # data : 토큰화, 고유번호 -> 단어들의 리스트
            # ei => data의 인덱스들
            # 한 단어씩 인코더에 넣어줌
            encoder_output, encoder_hidden = encoder(
                data[ei], encoder_hidden)
            # 인코더의 은닉상태를 저장
            encoder_outputs[ei] = encoder_output[0, 0]
        
        decoder_input = torch.tensor([[0]]).to(device)

        # 인코더의 마지막 은닉 상태를 디코더의 초기 은닉 상태로 지정
        decoder_hidden = encoder_hidden

        # 디코더 동작
        # 티처 포싱 (Teacher Forcing: 교사 강요)
        # Seq2Seq 구조에서 현시점의 입력을 (모델의 예측값을 사용하는 대신에) 정답을 이용하는 방법
        # 엉뚱한 답을 피하고, 시간 단축을 위해 강제적으로 정답을 넣어주는 기술 (50% 확률로 적용)
        use_teacher_forcing = True if random.random() < 0.5 else False
        
        if use_teacher_forcing:
            for di in range(len(label)):
                decorder_output = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)

                # 직접적으로 정답을 다음 시점의 입력으로 넣어줌
                target = torch.tensor(label[di], dtype=torch.long).to(device)
                target = torch.unsqueeze(target, dim=0).to(device)
                loss += nn.CrossEntropyLoss()(decorder_output, target)
                decorder_input = target
        else:
            for di in range(len(label)):
                decorder_output = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)

                # 가장 높은 확률을 갖는 단어의 인덱스 topi
                topv, topi = decorder_output.topk(1) # top k -> (1)개를 불러옴
                decorder_input = topi.squeeze().detach() # 텐서 -> 값

                # 디코더의 예측값을 다음 시점의 입력으로 넣어줌
                target = torch.tensor(label[di], dtype=torch.long).to(device)
                target = torch.unsqueeze(target, dim=0).to(device)
                loss += nn.CrossEntropyLoss()(decorder_output, target)
                
                if decoder_input.item() == 1: #<EOS> 토큰을 만나면 중지
                    break
        
        # 전체 손실 계산
        total_loss += loss.item() / len(dataset)
        iterator.set_description(f"epoch:{epoch+1} loss:{total_loss}")
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

torch.save(encoder.state_dict(), "attn_enc.pt")
torch.save(decoder.state_dict(), "attn_dec.pt")

  0%|          | 0/3592 [00:00<?, ?it/s]

<ipython-input-36-8559b5aca191>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=torch.long).to(device)
<ipython-input-36-8559b5aca191>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label, dtype=torch.long).to(device)
<ipython-input-36-8559b5aca191>:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(label[di], dtype=torch.long).to(device)
<ipython-input-36-8559b5aca191>:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone(

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

...

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

  0%|          | 0/3592 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
# https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_enc.pt
# https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_dec.pt
!wget https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_enc.pt
!wget https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_dec.pt

--2023-01-19 08:24:17--  https://github.com/bigdata-young/ai_26th/raw/main/etc/attn_enc.pt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bigdata-young/ai_26th/main/etc/attn_enc.pt [following]
--2023-01-19 08:24:18--  https://raw.githubusercontent.com/bigdata-young/ai_26th/main/etc/attn_enc.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 727147 (710K) [application/octet-stream]
Saving to: ‘attn_enc.pt’

attn_enc.pt         100%[===================>] 710.10K  --.-KB/s    in 0.04s   

2023-01-19 08:24:18 (15.8 MB/s) - ‘attn_enc.pt’ saved [727147/727147]

--2023-01-19 08:24:18--  https://github

# 성능 평가

In [ ]:
# 인코더 가중치 불러오기 # 인코더 파일과 디코더 파일 불러옴
encoder.load_state_dict(torch.load("attn_enc.pt", map_location=device))
decoder.load_state_dict(torch.load("attn_dec.pt", map_location=device))

<All keys matched successfully>

In [ ]:
# 불러올 영어 문장을 랜덤하게 지정
idx = random.randint(0, len(dataset))
# 테스트에 사용할 문장
input_sentence = dataset.eng_corpus[idx]
input_sentence

'i love red parrots'

In [ ]:
# 신경망이 번역한 문장
pred_sentence = ""

In [ ]:
data, label = dataset[idx]
data = torch.tensor(data, dtype=torch.long).to(device) # 영어문장
label = torch.tensor(label, dtype=torch.long).to(device) # 한국어문장

In [ ]:
data # 1은 문장이 종료 되었다는 의미

tensor([ 13, 225, 317, 717,   1], device='cuda:0')

In [ ]:
label

tensor([ 273, 1091, 1092,  575,    1], device='cuda:0')

## 인코더 동작

In [ ]:
# 인코더의 초기 은닉 상태 정의
encoder_hidden = torch.zeros(1, 1, 64).to(device)
# 인코더 출력을 담기 위한 변수
encoder_outputs = torch.zeros(11, 64).to(device)

In [ ]:
for ei in range(len(data)):
    # 한 단어씩 인코더에 넣어줌
    encoder_output, encoder_hidden = encoder(
        data[ei], encoder_hidden
    )
    # 인코더의 출력을 저장
    encoder_outputs[ei] = encoder_output[0,0]

In [ ]:
encoder_outputs

tensor([[ 0.0670, -0.0828,  0.1387,  0.4108,  0.3733, -0.4414,  0.0194, -0.2774,
         -0.5139, -0.2234, -0.1909, -0.2712, -0.6315, -0.2789, -0.3793, -0.2722,
          0.3381,  0.1874,  0.5575, -0.6125, -0.2118,  0.0347,  0.0246,  0.1240,
          0.1052, -0.8413,  0.5108,  0.1417,  0.7593, -0.0846,  0.3523, -0.0465,
          0.0349,  0.3018,  0.2236,  0.2729, -0.4056, -0.0851, -0.0676,  0.0128,
          0.4679, -0.3215,  0.3848, -0.3517, -0.1487,  0.7700,  0.6093, -0.0393,
         -0.0622, -0.5013,  0.3958, -0.2262,  0.9282,  0.0950,  0.0999, -0.5390,
          0.1563,  0.2714,  0.0749, -0.2940, -0.4514,  0.7106, -0.2316, -0.2117],
        [-0.1702, -0.5215, -0.3284,  0.1337,  0.5556, -0.0927,  0.0025,  0.4259,
          0.4903, -0.2767, -0.4070, -0.0807, -0.7873,  0.9292, -0.3033, -0.3583,
         -0.1113, -0.8033, -0.7059, -0.9154, -0.5492, -0.5493, -0.3327, -0.3343,
          0.4340, -0.8358,  0.0512, -0.1337,  0.7892, -0.6796,  0.4396,  0.1224,
          0.6009, -0.0141, 

## 디코더 동작

In [ ]:
# 디코더 초기 입력
decoder_input = torch.tensor([[0]]).to(device)
# 0 -> 문장이 시작되었다는 SOS 토큰

# 인코더의 마지막 은닉 상태 -> 디코더의 초기 은닉 상태 [둘에 연관점을 주고 싶어서]
decoder_hidden = encoder_hidden

In [ ]:
for di in range(11):
    # 디코더 모델을 통해서 단어별 나올 확률
    decoder_output = decoder(
        decorder_input, decoder_hidden, encoder_outputs
    )
    # 가장 높은 확률을 갖는 단어의 요소 계산
    topv, topi = decoder_output.topk(1)
    # 가장 높은 확률의 단어
    decoder_input = topi.squeeze().detach()

    # EOS 토큰을 만나면 중지
    if decoder_input.item() == 1:
        break
    
    # 예측 문자열에 가장 높은 확률의 단어를 추가
    pred_sentence += list(dataset.korBOW.keys())[decoder_input] + " "

In [ ]:
print(input_sentence)

i love red parrots


In [ ]:
print(pred_sentence)

## 통합

In [ ]:
import random
from tqdm.notebook import tqdm
from torch.optim.adam import Adam

# 학습에 사용할 프로세서 정의
device = "cuda" if torch.cuda.is_available() else 'cpu'
# 학습에 사용할 데이터셋
dataset = Eng2Kor()

# 인코더 디코더 정의
encoder = Encoder(input_size=len(dataset.engBOW), hidden_size=64).to(device)
decoder = Decoder(64, len(dataset.korBOW), dropout_p=0.1).to(device)

# 인코더 가중치 불러오기
encoder.load_state_dict(torch.load("attn_enc.pt", map_location=device))
# 디코더 가중치 불러오기
decoder.load_state_dict(torch.load("attn_dec.pt", map_location=device))

idx = random.randint(0, len(dataset))
# 테스트에 사용할 문장
input_sentence = dataset.eng_corpus[idx]
# 신경망이 번역한 문장
pred_sentence = ""

data, label = dataset[idx]
data = torch.tensor(data, dtype=torch.long).to(device)
label = torch.tensor(label, dtype=torch.long).to(device)

# ➋인코더의 초기 은닉 상태 정의
encoder_hidden = torch.zeros(1, 1, 64).to(device)
# 인코더 출력을 담기위한 변수
encoder_outputs = torch.zeros(11, 64).to(device)

for ei in range(len(data)):
   # ➊한 단어씩 인코더에 넣어줌
   encoder_output, encoder_hidden = encoder(
       data[ei], encoder_hidden)
     
   # ➋인코더의 출력을 저장
   encoder_outputs[ei] = encoder_output[0, 0]  

# ➌디코더의 초기 입력
# 0은 <SOS>토큰
decoder_input = torch.tensor([[0]]).to(device)

# ➍인코더의 마지막 은닉 상태를 디코더의 초기 은닉 상태로
decoder_hidden = encoder_hidden 

for di in range(11):
    # ➊가장 높은 확률을 갖는 단어의 요소를 구함
   decoder_output = decoder(
                       decoder_input, decoder_hidden, encoder_outputs)
   topv, topi = decoder_output.topk(1)
   decoder_input = topi.squeeze().detach()

   # ➋<EOS> 토큰을 만나면 중지
   if decoder_input.item() == 1:  
       break

   # ➌가장 높은 단어를 문자열에 추가
   pred_sentence += list(dataset.korBOW.keys())[decoder_input] + " "  

print(input_sentence)  # 영어 문장
print(pred_sentence)  # 한글 문장

its a pity
안타까워요 


In [ ]:
!pip install gtts -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 6.3 MB/s eta 0:00:00


In [ ]:
import random
from tqdm.notebook import tqdm
from torch.optim.adam import Adam

# 학습에 사용할 프로세서 정의
device = "cuda" if torch.cuda.is_available() else 'cpu'
# 학습에 사용할 데이터셋
dataset = Eng2Kor()

# 인코더 디코더 정의
encoder = Encoder(input_size=len(dataset.engBOW), hidden_size=64).to(device)
decoder = Decoder(64, len(dataset.korBOW), dropout_p=0.1).to(device)

# 인코더 가중치 불러오기
encoder.load_state_dict(torch.load("attn_enc.pt", map_location=device))
# 디코더 가중치 불러오기
decoder.load_state_dict(torch.load("attn_dec.pt", map_location=device))

idx = random.randint(0, len(dataset))
# 테스트에 사용할 문장
input_sentence = dataset.eng_corpus[idx]
# 신경망이 번역한 문장
pred_sentence = ""

data, label = dataset[idx]
data = torch.tensor(data, dtype=torch.long).to(device)
label = torch.tensor(label, dtype=torch.long).to(device)

# ➋인코더의 초기 은닉 상태 정의
encoder_hidden = torch.zeros(1, 1, 64).to(device)
# 인코더 출력을 담기위한 변수
encoder_outputs = torch.zeros(11, 64).to(device)

for ei in range(len(data)):
   # ➊한 단어씩 인코더에 넣어줌
   encoder_output, encoder_hidden = encoder(
       data[ei], encoder_hidden)
     
   # ➋인코더의 출력을 저장
   encoder_outputs[ei] = encoder_output[0, 0]  

# ➌디코더의 초기 입력
# 0은 <SOS>토큰
decoder_input = torch.tensor([[0]]).to(device)

# ➍인코더의 마지막 은닉 상태를 디코더의 초기 은닉 상태로
decoder_hidden = encoder_hidden 

for di in range(11):
    # ➊가장 높은 확률을 갖는 단어의 요소를 구함
   decoder_output = decoder(
                       decoder_input, decoder_hidden, encoder_outputs)
   topv, topi = decoder_output.topk(1)
   decoder_input = topi.squeeze().detach()

   # ➋<EOS> 토큰을 만나면 중지
   if decoder_input.item() == 1:  
       break

   # ➌가장 높은 단어를 문자열에 추가
   pred_sentence += list(dataset.korBOW.keys())[decoder_input] + " "  

from gtts import gTTS
from IPython.display import Audio
from time import sleep

file_name = '/content/sample.mp3'

text = input_sentence
tts_en = gTTS(text=text)
tts_en.save(file_name)

print(input_sentence)  # 영어 문장
wn = Audio(file_name, autoplay=True)
display(wn)

sleep(5)

text = pred_sentence
tts_ko = gTTS(text=text, lang='ko')
tts_ko.save(file_name)
print(pred_sentence)  # 한글 문장

wn = Audio(file_name, autoplay=True)
display(wn)

how many years did you live in boston


보스턴에서 동안 동안 동안 동안 살았어 


In [ ]:
# 한글 ->영어
# 티쳐포싱 수를 높이거나 다른 함수 사용해보기